In [9]:
import os
import pandas as pd

In [18]:
# 모든 document를 저장할 수 있는 text_set 선언
text_set = [] 

In [19]:
# open the folder where documents are saved, and save all the documents
path = 'data/'

for filename in os.listdir(path):
    file = open(os.path.join(path,filename), 'r', encoding='utf-8')
    text = file.readlines()
    text_set.append(text)

In [22]:
text_set[0]

[' (CNN)If you remember Miley twerking or Kanye\'s mic-snatching, you\'ll know music awards can get chaotic.Like on Tuesday, when singer Taylor Swift took a Twitter dig at hip-hop queen Nicki Minaj and the Internet erupted into a sprawling conversation about race and gender. And then Kim Kardashian got involved. Stay with us.\'Black women are rarely rewarded\'It started when Minaj took to her keyboard to vent after one of her biggest hits wasn\'t nominated for video of the year for the MTV Video Music Awards.Mind you, this isn\'t any old clip. "Anaconda," an uproarious, raunchy twist on pop culture portrayals of curvy black women, received 19.6 million clicks in its first 24 hours, a music video record at the time. So the rapper had a reason to be upset when she was snubbed -- and she made her reasoning clear."If I was a different \'kind\' of artist, Anaconda would be nominated for best choreo and vid of the year as well," said Minaj."Black women influence pop culture so much but are r

# TF-IDF Flow

### 1. 각 text별로 tokenize를 한다. 
### 2. token에서 stopwords나 필요없는 부분을 제거하고 stemming을 해준다. 
### 3. 각 document, 각 token별 token 몇개 있는지 구한 다음, table로 저장한다. 
### 4. 각 token별 어떤 document가 있는 지 inverted index를 구한다. 
### 5. query가 속한 document들만 가져온다. 
### 6. tf-idf를 구한다.
### 7. query의 tf-idf를 구한다. 
### 8. cosine-similarity를 구한다. 

# nltk

In [71]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer

In [35]:
# stopwords 불러오기.
stop = set(stopwords.words('english'))

In [38]:
stop.add('\'')

In [40]:
stop.add('\"')

In [69]:
stop.add("!")
stop.add("?")
stop.add("@")
stop.add("*")
stop.add("^")
stop.add("\"\"")
stop.add("\'\'")
stop.add("#")
stop.add("$")
stop.add("%")
stop.add("(")
stop.add(")")

In [74]:
vocab_list = []
porter_stemmer = PorterStemmer()

In [76]:
for i in range(len(text_set)):
    for j in range(len(text_set[i][0])):
        tokenized = word_tokenize(text_set[i][0].lower()) # 소문자로 만든 뒤 tokenized를 했다. 
        tokenized = [porter_stemmer.stem(token) for token in tokenized if token not in stop] # stopwords를 삭제했다. 
        vocab_list.append(tokenized)

KeyboardInterrupt: 

In [58]:
# vocab_set
vocab_set = set()

In [67]:
# token에서 stopwords를 제거하고 
for i in range(len(text_set)):
    for j in range(len(text_set[i][0])):
        tokenized = word_tokenize(text_set[i][0])
        for token in tokenized:
            if token not in stop:
                vocab_set.add(token)

KeyboardInterrupt: 

In [68]:
vocab_set

{'!',
 '#',
 '&',
 "''",
 "'90s",
 "'Black",
 "'Lemonade",
 "'kind",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 '(',
 ')',
 ',',
 '--',
 '.',
 '...',
 '1',
 '11',
 '11:30',
 '12',
 '14,000',
 '15',
 '19.6',
 '1992',
 '1997',
 '2009',
 '2012',
 '2015',
 '2017',
 '2018',
 '22.Beyoncé',
 '24',
 '25',
 '25th',
 '27',
 '30-something',
 '59th',
 '6',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 'A',
 'AND',
 'Abel',
 'Academy',
 'Adele',
 'Album',
 'All',
 'Anaconda',
 'And',
 'Angeles',
 'Anniversary',
 'April',
 'Arts',
 'As',
 'Ask',
 'At',
 'Awards',
 'Awards.Mind',
 'B',
 'Bad',
 'Beck',
 'Beef',
 'Before',
 'Bette',
 'Beyonce',
 'Beyonce.Though',
 'Beyoncé',
 'Beyoncé.Lady',
 'Big',
 'Black',
 'Blige',
 'Blood',
 'Bon',
 'Bowl',
 'British',
 'Bruno',
 'Butterfly',
 'CBS',
 'CNN',
 'Ca',
 'Can',
 'Canadian',
 'Center',
 'Chris',
 'Christina',
 'Coachella',
 'Coldplay',
 'Cologne',
 'Come',
 'Cool',
 'Country',
 'Crush',
 'Cue',
 'Cyrus',
 'Did',
 'Drake.But',
 'ET',
 'Ed',
 'Ehrlich.Last',
 'Entert